In [1]:
import tabula
import re
import pandas as pd

In [25]:
file_path = 'REP IN CONGRESS 2020 - 23rd DISTRICT.pdf'
output_file_name = 'tompkins_ny23_results_by_district_2020.csv'

# Convert PDF to DataFrame

In [3]:
df = tabula.read_pdf(file_path, pages='all', output_format='dataframe')

In [4]:
type(df)

list

In [5]:
data = df[0]

In [6]:
data.head()

,Unnamed: 0,Tracy\rMitrano,Tom Reed,Tom Reed.1,Tracy\rMitrano.1,Unnamed: 5,Andrew M.\rKolstee,Tom Reed.2,Unnamed: 8,BLANK\rUndervotes,VOID\rOvervotes,Unqualified Write\rins,Total Votes,Unnamed: 13
0,NaN,DEM,REP,CON,WOR,GRE,LIB,IND,SAM,NaN,NaN,NaN,NaN,NaN
1,Any,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,City of Ithaca 1 - 1,371,49,6,92,0,7,4,NaN,13.0,0.0,2.0,544.0,NaN
3,City of Ithaca 1 - 2,293,51,3,71,0,5,3,NaN,10.0,1.0,0.0,437.0,NaN
4,City of Ithaca 1 - 3,210,55,4,52,0,2,0,NaN,9.0,0.0,0.0,332.0,NaN


# Clean the DataFrame

Drop row index 1, it doesn't contain any data

In [7]:
data.drop(data.index[1], inplace=True)

In [8]:
data.head()

,Unnamed: 0,Tracy\rMitrano,Tom Reed,Tom Reed.1,Tracy\rMitrano.1,Unnamed: 5,Andrew M.\rKolstee,Tom Reed.2,Unnamed: 8,BLANK\rUndervotes,VOID\rOvervotes,Unqualified Write\rins,Total Votes,Unnamed: 13
0,NaN,DEM,REP,CON,WOR,GRE,LIB,IND,SAM,NaN,NaN,NaN,NaN,NaN
2,City of Ithaca 1 - 1,371,49,6,92,0,7,4,NaN,13.0,0.0,2.0,544.0,NaN
3,City of Ithaca 1 - 2,293,51,3,71,0,5,3,NaN,10.0,1.0,0.0,437.0,NaN
4,City of Ithaca 1 - 3,210,55,4,52,0,2,0,NaN,9.0,0.0,0.0,332.0,NaN
5,City of Ithaca 1 - 4,617,72,7,191,0,7,9,NaN,28.0,0.0,2.0,933.0,NaN


Drop Last Column, it contains no data

In [9]:
data.drop('Unnamed: 13', axis=1, inplace=True)

Clean up the column names

In [10]:
new_col_names = ['district', 'tracy_mitrano_dem', 'tom_reed_rep', 'tom_reed_con', 
                 'tracy_mitrano_wor', 'gre', 'andrew_kolstee_lib', 'tom_reed_ind', 
                 'sam', 'undervotes', 
                 'overvotes', 'write_ins', 'total_votes']

In [11]:
data.columns = new_col_names

Drop row 0, it is no longer needed

In [12]:
data.drop(data.index[0], inplace=True)
data = data.reset_index(drop=True)

In [13]:
data.head()

,district,tracy_mitrano_dem,tom_reed_rep,tom_reed_con,tracy_mitrano_wor,gre,andrew_kolstee_lib,tom_reed_ind,sam,undervotes,overvotes,write_ins,total_votes
0,City of Ithaca 1 - 1,371,49,6,92,0,7,4,NaN,13.0,0.0,2.0,544.0
1,City of Ithaca 1 - 2,293,51,3,71,0,5,3,NaN,10.0,1.0,0.0,437.0
2,City of Ithaca 1 - 3,210,55,4,52,0,2,0,NaN,9.0,0.0,0.0,332.0
3,City of Ithaca 1 - 4,617,72,7,191,0,7,9,NaN,28.0,0.0,2.0,933.0
4,City of Ithaca 2 - 1,529,39,7,196,0,4,3,NaN,28.0,0.0,1.0,807.0


Set the values for the sam column to be 0, for consistency

In [14]:
data['sam'] = 0

In [15]:
data.head()

,district,tracy_mitrano_dem,tom_reed_rep,tom_reed_con,tracy_mitrano_wor,gre,andrew_kolstee_lib,tom_reed_ind,sam,undervotes,overvotes,write_ins,total_votes
0,City of Ithaca 1 - 1,371,49,6,92,0,7,4,0,13.0,0.0,2.0,544.0
1,City of Ithaca 1 - 2,293,51,3,71,0,5,3,0,10.0,1.0,0.0,437.0
2,City of Ithaca 1 - 3,210,55,4,52,0,2,0,0,9.0,0.0,0.0,332.0
3,City of Ithaca 1 - 4,617,72,7,191,0,7,9,0,28.0,0.0,2.0,933.0
4,City of Ithaca 2 - 1,529,39,7,196,0,4,3,0,28.0,0.0,1.0,807.0


Convert vote count columns to integer

In [16]:
for col in data.columns[1:]:
    
    data[col] = pd.to_numeric(data[col], downcast='integer')

Add district abbreviation column

In [17]:
def get_dist_num(dist_name):
    """Get the district number from the district name. Whitespace must be 
       removed to match the style in the shapefile"""
    dist_num_list = re.findall(r'\d\s*-*\s*\d*', dist_name)
    if len(dist_num_list) == 1:
        return dist_num_list[0].replace(' ', '')
    else:
        return ''

In [18]:
def abv_dist_name(dist_name):
    """
    Convert a full district name to the abbreviation used in the associated
    shapefile.
    """
    dist_num = get_dist_num(dist_name)
    
    if 'City of Ithaca' in dist_name:
        return dist_num
    elif 'Caroline' in dist_name:
        return 'CAR' + '-' + dist_num
    elif 'Danby' in dist_name:
        return 'DAN' + '-' + dist_num
    elif 'Dryden' in dist_name:
        return 'DRY' + '-' + dist_num
    elif 'Enfield' in dist_name:
        return 'EN' + '-' + dist_num
    elif 'Groton' in dist_name:
        return 'GRT' + '-' + dist_num
    elif 'Town of Ithaca' in dist_name:
        return 'ITH' + '-' + dist_num
    elif 'Lansing' in dist_name:
        return 'LAN' + '-' + dist_num
    elif 'Newfield' in dist_name:
        return 'NEW' + '-' + dist_num
    elif 'Ulysses' in dist_name:
        return 'UL' + '-' + dist_num
    else:
        return 'error'

Create district abbreviation column

In [19]:
data['dist_abv'] = data['district'].apply(abv_dist_name)

Check for rows that weren't set to anything

In [20]:
data[data.dist_abv == '']

,district,tracy_mitrano_dem,tom_reed_rep,tom_reed_con,tracy_mitrano_wor,gre,andrew_kolstee_lib,tom_reed_ind,sam,undervotes,overvotes,write_ins,total_votes,dist_abv
8,"City of Ithaca 3 - 1, 3-2",445,38,5,91,0,5,4,0,11,0,2,601,


District 3-1 is very small, since it is combined with 3-2 in the pdf, it will be ignored.

In [21]:
data.loc[data.index[8], 'dist_abv'] = '3-2'

Lansing 2, 8 were also not set correctly, Lansing 8 will be ignored because it is relatively small

In [36]:
data[data['dist_abv'] == 'LAN-']

,district,tracy_mitrano_dem,tom_reed_rep,tom_reed_con,tracy_mitrano_wor,gre,andrew_kolstee_lib,tom_reed_ind,sam,undervotes,overvotes,write_ins,total_votes,dist_abv
50,"Town of Lansing 2, 8",400,336,17,32,0,15,11,0,30,0,1,842,LAN-


In [37]:
data.loc[data.index[50], 'dist_abv'] = 'LAN-8'

# Save output file

In [38]:
data.to_csv(output_file_name)